# Дообучение модели GPT-3 на корпусе текстов юридического характера

В юридических организациях проблема заключается в масштабе и усилиях. Объем документов в большинстве юридических организаций, даже крупных, далеко не тот, для которого были разработаны инструменты ИИ. Кроме того, проверка и сборка таких наборов данных, а также проверка работоспособности продукта после обучения на таких коллекциях могут занимать чрезвычайно много времени. 

Он поддерживает три основных варианта использования: семантический поиск, классификацию и генерацию текста.

Классификация протоколов аварийных ситуаций для страховой компании.

Берем готовую версию модели и дообучаем на корпусе текстов юр. характера.

1-й этап - раздобыть корпус тексов юридического характера. Для этого у нас есть:

* Министерство юстиции РФ (minjust.gov.ru) - 419 документов
* minjust-prikaz.consultant.ru - 912 документов

Можно использовать другие ресурсы.

2-й этап - дообучить.
Это требует ресурсы и понимание технологии.
Берем готовую модель и дообучаем.

Нужно понять, как запустить и как дообучить.

 Генерация текста — самая полезная и впечатляющая функция GPT-3: ее можно использовать для чего угодно: от создания полностью рабочего кода до постов в блогах и даже табулатуры для гитары. Примечательно, что программа обучается без какого-либо контроля со стороны человека. Получая множество различных данных из Интернета, включая Википедию, сообщения Reddit, всевозможные блоги, новостные статьи и даже фанфики, он находит шаблоны на основе статистических закономерностей, которые затем использует для выполнения текстовых подсказок.

Нейронная сеть следует набору правил для удовлетворения конкретных потребностей, и полоса креативности является одной из ее наиболее заметных настроек. Пользователи могут либо выбрать небольшую креативность, чтобы сузить алгоритм обучения до предоставленных материалов и получить очень структурированную и основанную на фактах информацию, либо установить высокую креативность и наблюдать, как ИИ делает свои собственные предположения и генерирует творческий (хотя и не обязательно правдивый) контент.

Благодаря своей замечательной способности обобщать и расширять содержание, модель может превратить длинный текст в краткий бриф и наоборот.

Классификация метаданных из любого юридического документа.

GPT-3 автоматизирует самую скучную часть юридической работы, такую ​​как заполнение запросов на допуск (RFA)

Ввод для GPT-3 не должен превышать 1000 символов.

Как правило, языковая модель, такая как GPT-3, адаптируется к конкретной задаче, такой как ответы на вопросы о документе или краткое изложение документа, с помощью метода, называемого «точной настройкой». Здесь вы изменяете архитектуру, чтобы заставить ее выполнять новую задачу, а затем продолжаете обучать ее на 100–1000 новых примерах, чтобы «настроить» ее на новую задачу.

Вместо этого в GPT-3 OpenAI использует более простую технику, которая настраивает модель на имитацию определенного поведения, предоставляя ей всего несколько примеров. Вес самой модели остается неизменным.

Удивительным результатом является то, что благодаря своему увеличенному размеру GPT-3 часто может приблизиться или соответствовать современным результатам, которые были достигнуты путем точной настройки на меньших моделях без точной настройки.

Мы можем быть уверены, что GPT-3 не видел приватных данных; это может означать, что он будет менее эффективен для типов документов, которые обычно не являются общедоступными.

Парсим сканы документов с сайта министерства юстиций.

Храним метаданные по документам в sqlite.

Распознаем текст со сканов. 

Учим GPT3 классифицировать юридические документы.

К счастью, API GPT-3 дает нам несколько ручек для настройки, чтобы почти полностью исключить предсказания за пределами допустимого диапазона. В этом сообщении блога мы рассмотрим два из них:

Регулировка temperature.
Установка logit_bias.
Регулировка температуры
Параметр temperatureдовольно простой. Документация GPT-3 объясняет, как temperatureэто работает: «Более высокие значения означают, что модель будет подвергаться большему риску. Попробуйте 0,9 для более творческих приложений и 0 (выборка argmax) для тех, у которых есть четко определенный ответ». Для задач классификации текста обычно требуется установить это значение равным 0.

Несмотря на то, что установка temperatureзначения 0 значительно уменьшает количество прогнозов, выходящих за пределы допустимого диапазона, она не устраняет их полностью. Чтобы сделать еще один шаг вперед, вы можете использовать logit_biasпараметр (подробнее об этом чуть позже).

Параметр logit_bias
Токенизация: как GPT-3 видит текст
Чтобы понять, как logit_biasможно использовать для создания белого списка меток, нам сначала нужно сделать шаг назад и объяснить, как GPT-3 интерпретирует текст. GPT-3 видит текст не как строку символов или слов, а как список «идентификаторов токенов». Идентификатор каждого токена — это уникальный номер, представляющий собой короткую последовательность символов (в среднем каждый токен соответствует примерно 4 символам английского текста). Процесс преобразования текста в токены основан на частоте определенных комбинаций символов. Распространенным словам, таким как «кошка» и «мяч», скорее всего, будет присвоен собственный уникальный токен, в то время как менее распространенные слова, такие как «учреждение», скорее всего, будут разделены на более чем один токен (например, «установить», «лишиться», «мент»).
https://medium.com/edge-analytics/getting-the-most-out-of-gpt-3-based-text-classifiers-part-one-797460a5556e

## Корпус текстов
Банк данных "Приказы и распоряжения Министерства юстиции Российской Федерации" - https://minjust-prikaz.consultant.ru/

Министерство юстиции Российской Федерации - https://minjust.gov.ru/ru/documents/

СудАкт: Судебные и нормативные акты РФСудебные и нормативные акты РФ - https://sudact.ru/regular/#doc_list

Государственная автоматизированная система Российской Федерации «Правосудие» (Информация о деятельности федеральных судов общей юрисдикции) - https://sudrf.ru/index.php?id=300

Архив решений арбитражных судов и судов общей юрисдикции - https://sudrf.cntd.ru/

На настоящий момент в России в открытом доступе находятся датасеты, созданные на основе решений судов по уголовным делам. Так, проект Достоевский содержит открытые данные об уголовных делах в России с 2009 года. Создатели проекта собирают, обрабатывают и визуализируют датасеты из официальной статистики Судебного департамента при Верховном суде РФ. - https://beta.dostoevsky.io/ru/full

 С 2011 года предполагается официальное опубликование правовых актов на Официальном интернет-портале правовой информации ( pravo.gov.ru ). Сейчас он включает и федеральное, и региональное, и муниципальное законодательство, но информация не полная. Однако такие документы бывают только в графическом формате (отсканированный TIFF или PDF без текстового слоя).
 
 https://www.clarin.eu/resource-families/legal-corpora

## Источники

Автодополнение текста на естественном языке c использованием глубокого обучения, Бакалаврская работа, Менчуков Павел Александрович, 